In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import random

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urljoin

from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

# 랜덤 대기 시간 설정
random_sec = random.uniform(1, 2)

# 웹드라이버 초기화 및 웹사이트 열기
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = "https://unipass.customs.go.kr/clip/index.do"
driver.get(url)

# 페이지가 완전히 로드될 때까지 기다림
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="TOPMENU_LNK_M_ULS0200000000"]/span')))

# 텍스트를 저장할 리스트 초기화
texts = []

hover_element = driver.find_element(By.XPATH, '//*[@id="TOPMENU_LNK_M_ULS0200000000"]/span')
click_element = driver.find_element(By.XPATH, '//*[@id="TOPMENU_LNK_UI-ULS-0201-002Q"]/span')

actions = ActionChains(driver)
actions.move_to_element(hover_element).perform()  # 마우스를 hover_element 위로 이동
actions.click(click_element).perform()  # click_element 클릭

time.sleep(random_sec)  # 페이지가 완전히 로드될 때까지 기다림

# 원하는 요소를 클릭
click_element = driver.find_element(By.XPATH, '/html/body/div[3]/section[2]/div[2]/div/div/form/div/table/tbody/tr[1]/td[2]/a')
click_element.click()

time.sleep(random_sec)  # 페이지가 완전히 로드될 때까지 기다림

for i in range(78, 98):
    xpath = f"/html/body/div[3]/section[2]/div[2]/div/form/div[2]/div/div/div/p[{i}]/a"
             # /html/body/div[3]/section[2]/div[2]/div/form/div[2]/div/div/div/p[2]/a : 호목록 페이지에서 클릭할 것
    try:
        p_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
        p_element.click()
    except StaleElementReferenceException:
        # 요소가 Stale 상태라면 다시 찾아서 클릭
        #p_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
        #p_element.click()
        p_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
        p_element.click()
        

    # 페이지 로딩 대기 및 필요한 작업 수행
    time.sleep(random_sec)
    # /html/body/div[3]/section[2]/div[2]/div/div/div/div[1]/table/tbody/tr[1]/td[2]/a : tr의 개수가 필요
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#tblLstBody tr')))
    soup = BeautifulSoup(driver.page_source, 'html.parser') # tr행만 soup변수에 할당
    rows = soup.select('#tblLstBody tr')    
#     # 데이터 수집
    for j in range(len(rows)):
        xpath2 = f"/html/body/div[3]/section[2]/div[2]/div/div/div/div[1]/table/tbody/tr[{j+1}]/td[2]/a"

        try:
            element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath2)))
            element.click()
            
        except StaleElementReferenceException:
            element = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath2)))
            element.click()
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#tblLstBody tr')))    
        time.sleep(random_sec)
        # 새 페이지의 HTML을 가져와 BeautifulSoup 객체를 생성
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 행의 수를 찾음
        num_rows = len(soup.select('#tblLstBody tr'))
        rows2 = soup.select('#tblLstBody tr')
        # 아주 잘뽑고 있음 print(rows2)  
    #for s in range(1, num_rows + 1):
        for row in rows2:
            number = row.select_one('td:nth-of-type(1) > a > span')
            number = number.text if number else ""

            sub_num = row.select_one('td:nth-of-type(2) a')
            sub_num = sub_num.text if sub_num else ""

            kor_name = row.select_one('td:nth-of-type(4) a')
            kor_name = kor_name.text if kor_name else ""

            eng_name = row.select_one('td:nth-of-type(5) a')
            eng_name = eng_name.text if eng_name else ""

            texts.append((number, sub_num, kor_name, eng_name))
    
        driver.back()
        time.sleep(random_sec)  # 페이지가 완전히 로드될 때까지 기다림
        
        
        
        try:
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath))).click()
        except StaleElementReferenceException:
        # 요소가 Stale 상태라면 다시 찾아서 클릭
        
            driver.execute_script("arguments[0].click();", p_element)
        time.sleep(random_sec)

# 결과 출력
print(texts)

[('7801', '\n\t\t\t\t\t\t\t\t\t\t\t\u3000\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t납의 괴(塊)\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\tUnwrought lead.\n\t\t\t\t\t\t\t\t\t\t'), ('7801', '\n\t\t\t\t\t\t\t\t\t\t\t10\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t정제한 납\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\tRefined lead\n\t\t\t\t\t\t\t\t\t\t'), ('7801', '\n\t\t\t\t\t\t\t\t\t\t\t10\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t납의 함유량이 전 중량의 100분의 99.99 이상인 것\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\tContaining by weight 99.99 % or more of lead\n\t\t\t\t\t\t\t\t\t\t'), ('7801', '\n\t\t\t\t\t\t\t\t\t\t\t10\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t기타\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\tOther\n\t\t\t\t\t\t\t\t\t\t'), ('7801', '\n\t\t\t\t\t\t\t\t\t\t\t9\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\t기타\n\t\t\t\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\t\t\t\tOther :\n\t\t\t\t\t\t\t\t\t\t'), ('7801', '\n\t\t\t\t\t\t\t\t\t\t\t91\n\t\t\t\t\t\t\t\t\t

In [ ]:
latest = texts # 첫 번째 크롤링 텍스트 변수 저장

In [16]:
latest2= texts # 두 번째 크롤링 텍스트 변수 저장

In [69]:
# 텍스트 전처리
cleaned_texts = [(a.replace('\t', '').replace('\n', '').replace('\u3000', ' '),
                  b.replace('\t', '').replace('\n', '').replace('\u3000', ' '),
                  c.replace('\t', '').replace('\n', '').replace('\u3000', ' '),
                  d.replace('\t', '').replace('\n', '').replace('\u3000', ' ')) for a, b, c, d in latest]

In [17]:
# 텍스트 전처리
cleaned_texts2 = [(a.replace('\t', '').replace('\n', '').replace('\u3000', ' '),
                  b.replace('\t', '').replace('\n', '').replace('\u3000', ' '),
                  c.replace('\t', '').replace('\n', '').replace('\u3000', ' '),
                  d.replace('\t', '').replace('\n', '').replace('\u3000', ' ')) for a, b, c, d in latest2]

In [18]:
# df 변환
import pandas as pd

df2 = pd.DataFrame(cleaned_texts2, columns=['호','부호', '한글명', '영문명'],dtype=str)

In [35]:
# df 결합
df_final = pd.concat([df,df2])

In [40]:
df_final.to_csv('HSCODE_소호.csv', index=False)